In this notebook, our goal is to select the top N articles with the highest proportion of each distinct topic. This allows us to explore the key discussions associated with each topic.

First, we load the datasets from Handelsblatt, SZ, Welt, and dpa.

In [1]:
import os
import pandas as pd
from ast import literal_eval
from datetime import datetime
startTime = datetime.now()

# Set the path variable to point to the 'newspaper_data_processing' directory
path = os.getcwd().replace('\\newspaper_analysis\\topics', '\\newspaper_data_processing')

# Load pre-processed 'dpa' dataset from a CSV file
dpa = pd.read_csv(path + '\\dpa\\' + 'dpa_prepro_final.csv', encoding = 'utf-8', sep=';', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

# Keep only the article texts and their respective publication dates
dpa = dpa[['texts', 'day', 'month', 'year']]

# Load pre-processed 'SZ' dataset from a CSV file
sz = pd.read_csv(path + '\\SZ\\' + 'sz_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'newspaper_2': 'str',
                                                                                                 'quelle_texts': 'str',
                                                                                                 'page': 'str',
                                                                                                 'rubrics': 'str'})
sz.page = sz.page.fillna('')
sz.newspaper = sz.newspaper.fillna('')
sz.newspaper_2 = sz.newspaper_2.fillna('')
sz.rubrics = sz.rubrics.fillna('')
sz.quelle_texts = sz.quelle_texts.fillna('')

# Keep only the article texts and their respective publication dates
sz = sz[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Handelsblatt' dataset from a CSV file
hb = pd.read_csv(path + '\\Handelsblatt\\' + 'hb_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'kicker': 'str',
                                                                                                 'page': 'str',
                                                                                                 'series_title': 'str',
                                                                                                 'rubrics': 'str'})
hb.page = hb.page.fillna('')
hb.series_title = hb.series_title.fillna('')
hb.kicker = hb.kicker.fillna('')
hb.rubrics = hb.rubrics.fillna('')

# Keep only the article texts and their respective publication dates
hb = hb[['texts', 'day', 'month', 'year']]

# Load pre-processed 'Welt' dataset from a CSV file
welt = pd.read_csv(path + '\\Welt\\' + 'welt_prepro_final.csv', encoding = 'utf-8-sig', sep=';', index_col = 0, dtype = {'newspaper': 'str',
                                                                                                 'rubrics': 'str',
                                                                                                 'title': 'str'})
welt.title = welt.title.fillna('')
welt.rubrics = welt.rubrics.fillna('')

# Keep only the article texts and their respective publication dates
welt = welt[['texts', 'day', 'month', 'year']]

# Concatenate the 'dpa', 'sz', 'hb', and 'welt' DataFrames into a single DataFrame 'data'
data = pd.concat([dpa, sz, hb, welt])

# The number of articles in the final dataset
print(len(data))

# Sort the data in chronological order
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True])
# Reset the index of the DataFrame
data.reset_index(inplace=True, drop=True)
data.head()

3336299


,texts,day,month,year
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,1,1,1991
1,Welajati: Iran bleibt bei einem Krieg am Golf ...,1,1,1991
2,Bush will offenbar seinen Außenminister erneut...,1,1,1991
3,Sperrfrist 1. Januar 1000 HBV fordert umfassen...,1,1,1991
4,Schamir weist Nahost-Äußerungen des neuen EG-P...,1,1,1991


Next, we import sentiment scores, previously computed using an LSTM model for each article in the corpus.

In [2]:
import csv
import codecs

# Set the path variable to point to the 'sentiment' directory
path = os.getcwd().replace('\\topics', '') + '\\sentiment'

with codecs.open(path + "\\scores_lstm.csv", "r", encoding='utf-8-sig') as f:
    reader = csv.reader(f)
    scores = [None if row[0] == '' else float(row[0]) for row in reader]   

We add sentiment scores as a new column to the `data` DataFrame and discard any rows with missing sentiment scores.

In [3]:
# Add the sentiment scores as a new column in the data DataFrame
data['scores'] = scores

# Remove any rows in the DataFrame where a sentiment score is missing (NaN). In this context, 
# NaN corresponds to the model's inability to predict sentiment for certain 
# articles due to formatting issues or because the article is too short (less than 20 tokens).
data = data.dropna(subset=['scores'])

# Reset the index of the DataFrame
data.reset_index(inplace=True, drop=True)

Afterward, we are incorporating the topic distributions for each article, which were previously computed using the Latent Dirichlet Allocation (LDA) algorithm in the notebook named `Topic model estimation`. 

In [4]:
# Load the article topics from a CSV file.
article_topics = pd.read_csv('article_topic.csv', encoding='utf-8', index_col=0)

# Merge the `data` DataFrame with the `article_topics` DataFrame
data = pd.concat([data, article_topics], axis=1)

Now we identify the top `n` articles with the highest proportion of a certain topic, and for each of these articles, we print the date of publication, the text, the proportion of this topic in the article, the sentiment score, and the sentiment sign (which is '+1' for scores greater than or equal to 0.5, and '-1' otherwise).

In [5]:
# Set the number of articles you want to pick
n = 1

# Set the topic of interest
topic = 'T27'

# Sort the dataframe by the topic in descending order and pick the top n articles
top_articles = data.sort_values(by=[topic], ascending=False).head(n)

# Print the texts of the top articles for the topic, their proportion for that topic, and their score
for i, row in top_articles.iterrows():
    print(f'Article {i}:')
    print('Date of Publication:', f"{row['day']}-{row['month']}-{row['year']}")
    print('Text:', row['texts'])
    print('Topic proportion:', row[topic])
    print('Score:', row['scores'])
    print('Sign:', '+1' if row['scores'] >= 0.5 else '-1')
    print('\n---\n')

Article 592093:
Date of Publication: 15-5-2009
Text: Deutsche Wirtschaft abgestürzt - Tiefpunkt erreicht?   (Mit Grafik 10829 und Bildern). Wiesbaden Dieser Absturz ist beispiellos: Nach fünf Boomjahren in Folge hat die globale Wirtschaftskrise deutlich spürbare Kratzer an Deutschlands Wirtschaft hinterlassen. Das Land steckt in der tiefsten Rezession der Nachkriegsgeschichte. Noch nie seit Beginn der Quartals-Erhebungen durch das Statistische Bundesamt im Jahr 1970 brach das Bruttoinlandsprodukt (BIP) so rasant ein wie im ersten Quartal 2009. «Das Bruttoinlandsprodukt ist gegenüber dem vierten Quartal 2008 um 3,8 Prozent kollabiert», schreibt Commerzbank-Chefvolkswirt Jörg Krämer. Volkswirt Stefan Bielmeier von der Deutschen Bank stellt fest: «Einen solchen Einbruch, der sich über vier Quartale beschleunigt, gab es noch nie. Nicht mal bei der letzten Ölkrise.» Schon ist von einer «Schockstarre» der Wirtschaft die Rede, die NordLB spricht von «Horrorzahlen». So schnell wird der Wachstu

We also identify the top `n` articles that have a sentiment score less than 0.2 (indicating negative sentiment) and the highest proportion of a certain topic. This allows us to explore articles that extensively discuss this topic and convey a negative sentiment.

In [6]:
# Set the number of articles you want to pick
n = 1

# Set the topic of interest
topic = 'T81'

# Filter articles with score less than 0.2
filtered_data = data[data['scores'] < 0.2]

# Sort the filtered dataframe by the topic in descending order and pick the top n articles
top_articles = filtered_data.sort_values(by=[topic], ascending=False).head(n)

# Print the texts of the top articles for the topic, their proportion for that topic, and their score
for i, row in top_articles.iterrows():
    print(f'Article {i + 1}:')
    print('Date of Publication:', f"{row['day']}-{row['month']}-{row['year']}")
    print('Text:', row['texts'])
    print('Topic proportion:', row[topic])
    print('Score:', row['scores'])
    print('Sign:', '+1' if row['scores'] >= 0.5 else '-1')
    print('\n---\n')

Article 611481:
Date of Publication: 26-11-2009
Text: (Zusammenfassung 1900) Länder fordern Klarheit über Opel-Stellenabbau (Mit Bildern). Rüsselsheim Scharfe Kritik aus den Bundesländern mit Opel- Standorten an General Motors: Die derzeitigen Spekulationen über den Stellenabbau seien «unerträglich», klagte Hessens Ministerpräsident Roland Koch (CDU) am Donnerstag in Wiesbaden. GM solle endlich Klarheit über die Stellenstreichungen in den Opel-Werken schaffen. Koch bezog sich vor allem auf die Zahl von 2500 wegfallenden Stellen im Rüsselsheimer Stammwerk, die inakzeptabel sei. Er habe sie nur «aus der Zeitung erfahren». Thüringens Ministerpräsidentin Christine Lieberknecht (CDU) forderte ein schriftliches Sanierungskonzept von GM. Vorher habe es keinen Sinn, über öffentliche Hilfen zu diskutieren. «Wir wollen detailliert wissen, was passiert.» Lieberknecht will Garantien von General Motors (GM) für den Erhalt der vier deutschen Opel-Werke. Die Zusagen, die GM gemacht hat, müssten «schw

Moreover, it can be insightful to identify articles from a specific year (e.g., a crisis year like 2008) that have the highest share of a selected topic and a negative sentiment (score less than 0.2).

In [7]:
# Set the number of articles you want to pick
n = 1

# Set the topic of interest
topic = 'T74'

# Filter articles from the year 2008 with a sentiment score less than 0.2 (indicating negative sentiment)
filtered_data = data[(data['year']==2008) & (data['scores'] < 0.2)]

# Sort the filtered dataframe by the topic in descending order and pick the top n articles
top_articles = filtered_data.sort_values(by=[topic], ascending=False).head(n)

# Print the texts of the top articles for the topic, their proportion for that topic, and their score
for i, row in top_articles.iterrows():
    print(f'Article {i + 1}:')
    print('Date of Publication:', f"{row['day']}-{row['month']}-{row['year']}")
    print('Text:', row['texts'])
    print('Topic proportion:', row[topic])
    print('Score:', row['scores'])
    print('Sign:', '+1' if row['scores'] >= 0.5 else '-1')
    print('\n---\n')

Article 552228:
Date of Publication: 16-5-2008
Text: US-ZENTRALBANK. Blasenleiden. TORSTEN RIECKE Als Alan Greenspan 2002 gefragt wurde, ob das Platzen der Internetblase ein heilsamer Schock gewesen sei, versicherte der damalige US-Notenbankchef, dass es lange dauern würde, bevor sich die Anleger wieder von einem derartigen irrationalen Überschwang fortreißen ließen. Ein folgenschwerer Irrtum des Maestros der Geldpolitik. Nur wenige Jahre später bildete sich in den USA die nächste Spekulationsblase, diesmal auf dem Immobilienmarkt. Die Folgen bezeichnet der inzwischen pensionierte Greenspan als größtes Finanzdesaster seit dem Ende des Zweiten Weltkriegs. Dass seine lockere Geldpolitik zwischen 2001 und 2006 die Blase erst aufgepumpt hat, bestreitet er energisch. Dennoch hat die schnelle Folge von zwei riesigen Spekulationswellen die Notenbanker weltweit nachdenklich gemacht. Sogar bei der Fed in Washington wird jetzt darüber diskutiert, ob die Zentralbanker nicht aktiver gegen Exzesse 

Or, we can identify articles from a specific year with the highest share of a selected topic to understand the key events covered at that time within the context of that topic.

In [8]:
# Set the number of articles you want to pick
n = 1

# Set the topic of interest
topic = 'T100'

# Filter articles from the year 2007
filtered_data = data[data['year']==2007]

# Sort the filtered dataframe by the topic in descending order and pick the top n articles
top_articles = filtered_data.sort_values(by=[topic], ascending=False).head(n)

# Print the texts of the top articles for the topic, their proportion for that topic, and their score
for i, row in top_articles.iterrows():
    print(f'Article {i + 1}:')
    print('Date of Publication:', f"{row['day']}-{row['month']}-{row['year']}")
    print('Text:', row['texts'])
    print('Topic proportion:', row[topic])
    print('Score:', row['scores'])
    print('Sign:', '+1' if row['scores'] >= 0.5 else '-1')
    print('\n---\n')

Article 530762:
Date of Publication: 8-10-2007
Text: Rezessionsgefahr in den USA sinkt. Überraschend positive Nachrichten vom Arbeitsmarkt dämpfen Konjunktursorgen. FRANKFURT.Der amerikanische Arbeitsmarkt hat im September für eine positive Überraschung gesorgt. Zum einen sind im vergangenen Monat in den USA deutlich mehr neue Stellen entstanden als erwartet, zum anderen revidierten die Statistiker die bislang schwachen August-Zahlen stark nach oben. Die Finanzmärkte reagierten überaus positiv auf diese Nachrichten, weil die Sorgen vor einer Rezession in der US-Wirtschaft einen deutlichen Dämpfer erhielten. Als die Zahlen bekannt gegeben wurden, "ging ein Raunen durch den Händlersaal der Commerzbank", schreibt Jörg Krämer, Chefökonom des Finanzhauses, in seinem Handelsblatt-Weblog. Die Zahl der Beschäftigten außerhalb der Landwirtschaft stieg nach Angaben des US-Arbeitsministeriums im September um 110 000. Zudem entwickelte sich der Arbeitsmarkt in den Vormonaten deutlich besser als bi